# Neural networks

In [1]:
# Data handling and visualization
import pandas as pd
import torch

# Scale Data
from sklearn.preprocessing import StandardScaler

# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

# Models
from torch import nn

# Model evaluation
from sklearn import metrics

# Confusion matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

/tmp/ipykernel_327443/489267007.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data = pd.read_csv("../results/cardiovascular_disease_clean.csv")
data

,Age(years),Gender,Height(cm),Weight(kg),SystolicPressure,DiastolicPressure,Cholesterol,Glucose,Smoke,Alcohol,Active,cardio_disease,BMI,Pulse
0,50.391781,Male,168,62.0,110,80,Normal,Normal,No,No,Yes,No,21.967120,30
1,55.419178,Female,156,85.0,140,90,Well-Above-Normal,Normal,No,No,Yes,Yes,34.927679,50
2,51.663014,Female,165,64.0,130,70,Well-Above-Normal,Normal,No,No,No,Yes,23.507805,60
3,48.282192,Male,169,82.0,150,100,Normal,Normal,No,No,Yes,Yes,28.710479,50
4,47.873973,Female,156,56.0,100,60,Normal,Normal,No,No,No,No,23.011177,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67235,57.736986,Female,165,80.0,150,80,Normal,Normal,No,No,Yes,Yes,29.384757,70
67236,52.712329,Male,168,76.0,120,80,Normal,Normal,Yes,No,Yes,No,26.927438,40
67237,52.235616,Male,183,105.0,180,90,Well-Above-Normal,Normal,No,Yes,No,Yes,31.353579,90
67238,61.454795,Female,163,72.0,135,80,Normal,Above-Normal,No,No,No,Yes,27.099251,55


In [3]:
for col in ["Smoke", "Alcohol", "Active", "cardio_disease"]:
    data[col] = data[col].map({'No': 0, 'Yes': 1})

for col in ["Cholesterol", "Glucose"]:
    data[col] = data[col].map({'Normal': 1, 'Above-Normal': 2, 'Well-Above-Normal':3})

# We previously found that gender and height doesn't apport information we discard it
train_data = data.drop(columns=["Gender", "Height(cm)", "cardio_disease"])
cardio_disease = data.loc[:,"cardio_disease"]

train_data

,Age(years),Weight(kg),SystolicPressure,DiastolicPressure,Cholesterol,Glucose,Smoke,Alcohol,Active,BMI,Pulse
0,50.391781,62.0,110,80,1,1,0,0,1,21.967120,30
1,55.419178,85.0,140,90,3,1,0,0,1,34.927679,50
2,51.663014,64.0,130,70,3,1,0,0,0,23.507805,60
3,48.282192,82.0,150,100,1,1,0,0,1,28.710479,50
4,47.873973,56.0,100,60,1,1,0,0,0,23.011177,40
...,...,...,...,...,...,...,...,...,...,...,...
67235,57.736986,80.0,150,80,1,1,0,0,1,29.384757,70
67236,52.712329,76.0,120,80,1,1,1,0,1,26.927438,40
67237,52.235616,105.0,180,90,3,1,0,1,0,31.353579,90
67238,61.454795,72.0,135,80,1,2,0,0,0,27.099251,55


In [4]:
# Standardize data
std_scaler = StandardScaler()
std_data = std_scaler.fit_transform(train_data.values)

# Split the raw data into training and testing sets 
std_train_features, std_test_features, train_labels, test_labels = train_test_split(std_data, cardio_disease.values, test_size = 0.2, random_state = 42)
print("\nStandardized data")
print('Training Features Shape:', std_train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', std_test_features.shape)
print('Testing Labels Shape:', test_labels.shape)


Standardized data
Training Features Shape: (53792, 11)
Training Labels Shape: (53792,)
Testing Features Shape: (13448, 11)
Testing Labels Shape: (13448,)


In [5]:
# Define device
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Device to use: {device}")

# Convert to tensors
train_features_tensor = torch.tensor(std_train_features, dtype=torch.float32)
test_features_tensor = torch.tensor(std_test_features, dtype=torch.float32)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.long)

# DataLoader
batch_size = 64
train_dataset = torch.utils.data.TensorDataset(train_features_tensor, train_labels_tensor)
test_dataset = torch.utils.data.TensorDataset(test_features_tensor, test_labels_tensor)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Train function
def train(dataloader, neural_net, loss_fn, optimizer):
    size = len(dataloader.dataset)
    neural_net.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = neural_net(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# Función de prueba
def test(dataloader, neural_net, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    neural_net.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = neural_net(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    # print(f"Fase de prueba:")
    print(f"Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}")

Device to use: cpu


## NN classes def

In [6]:
class Shallow(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(11, 512),  # 11 input characteristics
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 2)  # 2 output classes
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [7]:
class ShallowThin(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(11, 10),  # 11 input characteristics
            nn.ReLU(),
            nn.Linear(10, 10),
            nn.ReLU(),
            nn.Linear(10, 2)  # 2 output classes
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [8]:
# Feed Forward with Full Connected layers
class FeedForwardFullConnected(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        
        input_size = 11
        hidden_size = 400
        output_size = 2

        self.linear_relu_stack = nn.Sequential(
            # 11 input layer characteristics
            nn.Linear(input_size, hidden_size), 
            nn.ReLU(), # Hidden layer 1
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(), # Hidden layer 2
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(), # Hidden layer 3
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(), # Hidden layer 4
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(), # Hidden layer 5
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(), # Hidden layer 6
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(), # Hidden layer 7
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(), # Hidden layer 8
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(), # Hidden layer 9
            nn.Linear(hidden_size, output_size) 
            # 2 output classes
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


In [ ]:
# Feed Forward with Full Connected layers
class FeedForwardFullConnected(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        
        input_size = 11
        hidden_size = 400
        output_size = 2

        self.linear_relu_stack = nn.Sequential(
            # 11 input layer characteristics
            nn.Linear(input_size, hidden_size), 
            nn.ReLU(), # Hidden layer 1
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(), # Hidden layer 2
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(), # Hidden layer 3
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(), # Hidden layer 4
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(), # Hidden layer 5
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(), # Hidden layer 6
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(), # Hidden layer 7
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(), # Hidden layer 8
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(), # Hidden layer 9
            nn.Linear(hidden_size, output_size) 
            # 2 output classes
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [9]:
# Feed Forward with Random Partial Connected layers

class RandomlyPartiallyConnectedLayer(nn.Module):
    def __init__(self, in_features, out_features, connection_ratio):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(out_features, in_features))
        self.bias = nn.Parameter(torch.randn(out_features))
        
        # Random connectivity mask (connection_ratio %)
        self.connection_mask = (torch.rand(out_features, in_features) < connection_ratio).float()
        self.connection_mask = nn.Parameter(self.connection_mask, requires_grad=False)

    def forward(self, x):
        # Apply mask to weight
        masked_weight = self.weight * self.connection_mask
        return torch.matmul(x, masked_weight.T) + self.bias


class FeedForwardRandomPartiallyConnected(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.ModuleList()

        input_size = 11
        hidden_size = 400
        num_hidden_layers = 9
        connection_ratio = 0.7
        output_size = 2
        
        # Input layer (fully connected)
        self.layers.append(nn.Linear(input_size, hidden_size))
        
        # Hidden layers (partially connected)
        for _ in range(num_hidden_layers - 1):
            self.layers.append(
                RandomlyPartiallyConnectedLayer(hidden_size, hidden_size, connection_ratio)
            )
        
        # Output layer (fully connected)
        self.layers.append(nn.Linear(hidden_size, output_size))
        self.activation = nn.ReLU()

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = self.activation(layer(x))
        # Última capa sin activación (para logits)
        x = self.layers[-1](x)
        return x


In [10]:
# Autoencoder with Wasp Waist
class AutoencoderWaspWaist(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        
        input_size = 11
        hidden_sizes = [400, 240, 160, 104, 64, 104, 160, 240, 400]
        output_size = 2

        self.linear_relu_stack = nn.Sequential(
            # 11 input layer characteristics
            nn.Linear(input_size, hidden_sizes[0]), 
            nn.ReLU(), # Hidden layer 1
            nn.Linear(hidden_sizes[0], hidden_sizes[1]),
            nn.ReLU(), # Hidden layer 2
            nn.Linear(hidden_sizes[1], hidden_sizes[2]),
            nn.ReLU(), # Hidden layer 3
            nn.Linear(hidden_sizes[2], hidden_sizes[3]),
            nn.ReLU(), # Hidden layer 4
            nn.Linear(hidden_sizes[3], hidden_sizes[4]),
            nn.ReLU(), # Hidden layer 5
            nn.Linear(hidden_sizes[4], hidden_sizes[5]),
            nn.ReLU(), # Hidden layer 6
            nn.Linear(hidden_sizes[5], hidden_sizes[6]),
            nn.ReLU(), # Hidden layer 7
            nn.Linear(hidden_sizes[6], hidden_sizes[7]),
            nn.ReLU(), # Hidden layer 8
            nn.Linear(hidden_sizes[7], hidden_sizes[8]),
            nn.ReLU(), # Hidden layer 9
            nn.Linear(hidden_sizes[8], output_size) 
            # 2 output classes
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [15]:
# Autoencoder with Wasp Waist
class SmallAutoencoderWaspWaist(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        
        input_size = 11
        hidden_sizes = [22, 20, 15, 10, 7, 10, 15, 20, 22]
        output_size = 2

        self.linear_relu_stack = nn.Sequential(
            # 11 input layer characteristics
            nn.Linear(input_size, hidden_sizes[0]), 
            nn.ReLU(), # Hidden layer 1
            nn.Linear(hidden_sizes[0], hidden_sizes[1]),
            nn.ReLU(), # Hidden layer 2
            nn.Linear(hidden_sizes[1], hidden_sizes[2]),
            nn.ReLU(), # Hidden layer 3
            nn.Linear(hidden_sizes[2], hidden_sizes[3]),
            nn.ReLU(), # Hidden layer 4
            nn.Linear(hidden_sizes[3], hidden_sizes[4]),
            nn.ReLU(), # Hidden layer 5
            nn.Linear(hidden_sizes[4], hidden_sizes[5]),
            nn.ReLU(), # Hidden layer 6
            nn.Linear(hidden_sizes[5], hidden_sizes[6]),
            nn.ReLU(), # Hidden layer 7
            nn.Linear(hidden_sizes[6], hidden_sizes[7]),
            nn.ReLU(), # Hidden layer 8
            nn.Linear(hidden_sizes[7], hidden_sizes[8]),
            nn.ReLU(), # Hidden layer 9
            nn.Linear(hidden_sizes[8], output_size) 
            # 2 output classes
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [11]:
# Autoencoder with Bottle Neck
class AutoencoderBottleNeck(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        
        input_size = 11
        hidden_sizes = [400, 240, 160, 104, 104, 104, 160, 240, 400]
        output_size = 2

        self.linear_relu_stack = nn.Sequential(
            # 11 input layer characteristics
            nn.Linear(input_size, hidden_sizes[0]), 
            nn.ReLU(), # Hidden layer 1
            nn.Linear(hidden_sizes[0], hidden_sizes[1]),
            nn.ReLU(), # Hidden layer 2
            nn.Linear(hidden_sizes[1], hidden_sizes[2]),
            nn.ReLU(), # Hidden layer 3
            nn.Linear(hidden_sizes[2], hidden_sizes[3]),
            nn.ReLU(), # Hidden layer 4
            nn.Linear(hidden_sizes[3], hidden_sizes[4]),
            nn.ReLU(), # Hidden layer 5
            nn.Linear(hidden_sizes[4], hidden_sizes[5]),
            nn.ReLU(), # Hidden layer 6
            nn.Linear(hidden_sizes[5], hidden_sizes[6]),
            nn.ReLU(), # Hidden layer 7
            nn.Linear(hidden_sizes[6], hidden_sizes[7]),
            nn.ReLU(), # Hidden layer 8
            nn.Linear(hidden_sizes[7], hidden_sizes[8]),
            nn.ReLU(), # Hidden layer 9
            nn.Linear(hidden_sizes[8], output_size) 
            # 2 output classes
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [17]:
# Loss function
loss_fn = nn.CrossEntropyLoss()

# Epochs
epochs = 15

In [18]:
# Instance Shallow
s_nn = Shallow().to(device)
optimizer = torch.optim.AdamW(s_nn.parameters(), lr=1e-3)

# Train model
for t in range(epochs):
    print(f"--------------------\nEpoch {t+1}")
    train(train_dataloader, s_nn, loss_fn, optimizer)
    test(test_dataloader, s_nn, loss_fn)

--------------------
Epoch 1
Accuracy: 73.0%, Avg loss: 0.549330
--------------------
Epoch 2
Accuracy: 72.7%, Avg loss: 0.549786
--------------------
Epoch 3
Accuracy: 73.1%, Avg loss: 0.546210
--------------------
Epoch 4
Accuracy: 73.3%, Avg loss: 0.544955
--------------------
Epoch 5
Accuracy: 73.2%, Avg loss: 0.542357
--------------------
Epoch 6
Accuracy: 73.2%, Avg loss: 0.544953
--------------------
Epoch 7
Accuracy: 73.1%, Avg loss: 0.544213
--------------------
Epoch 8
Accuracy: 73.2%, Avg loss: 0.542937
--------------------
Epoch 9
Accuracy: 73.1%, Avg loss: 0.543608
--------------------
Epoch 10
Accuracy: 73.2%, Avg loss: 0.544837
--------------------
Epoch 11
Accuracy: 73.2%, Avg loss: 0.548053
--------------------
Epoch 12
Accuracy: 72.8%, Avg loss: 0.551268
--------------------
Epoch 13
Accuracy: 73.0%, Avg loss: 0.544539
--------------------
Epoch 14
Accuracy: 73.3%, Avg loss: 0.544218
--------------------
Epoch 15
Accuracy: 73.1%, Avg loss: 0.547012


In [19]:
# Instance Shallow Thin
st_nn = ShallowThin().to(device)
optimizer = torch.optim.AdamW(st_nn.parameters(), lr=1e-3)

# Train model
for t in range(epochs):
    print(f"--------------------\nEpoch {t+1}")
    train(train_dataloader, st_nn, loss_fn, optimizer)
    test(test_dataloader, st_nn, loss_fn)

--------------------
Epoch 1
Accuracy: 73.1%, Avg loss: 0.546960
--------------------
Epoch 2
Accuracy: 73.1%, Avg loss: 0.544016
--------------------
Epoch 3
Accuracy: 73.1%, Avg loss: 0.543024
--------------------
Epoch 4
Accuracy: 73.2%, Avg loss: 0.542186
--------------------
Epoch 5
Accuracy: 73.1%, Avg loss: 0.542380
--------------------
Epoch 6
Accuracy: 73.1%, Avg loss: 0.543125
--------------------
Epoch 7
Accuracy: 73.2%, Avg loss: 0.542827
--------------------
Epoch 8
Accuracy: 73.0%, Avg loss: 0.541829
--------------------
Epoch 9
Accuracy: 72.9%, Avg loss: 0.542291
--------------------
Epoch 10
Accuracy: 72.9%, Avg loss: 0.542536
--------------------
Epoch 11
Accuracy: 73.1%, Avg loss: 0.542746
--------------------
Epoch 12
Accuracy: 73.1%, Avg loss: 0.541731
--------------------
Epoch 13
Accuracy: 73.2%, Avg loss: 0.543020
--------------------
Epoch 14
Accuracy: 73.0%, Avg loss: 0.541588
--------------------
Epoch 15
Accuracy: 73.0%, Avg loss: 0.541912


In [21]:
# Instance Feed Forward with Full Connected
fffc_nn = FeedForwardFullConnected().to(device)
optimizer = torch.optim.AdamW(fffc_nn.parameters(), lr=1e-3)

# Train model
for t in range(epochs):
    print(f"--------------------\nEpoch {t+1}")
    train(train_dataloader, fffc_nn, loss_fn, optimizer)
    test(test_dataloader, fffc_nn, loss_fn)

--------------------
Epoch 1
Accuracy: 72.7%, Avg loss: 0.547072
--------------------
Epoch 2
Accuracy: 73.2%, Avg loss: 0.549218
--------------------
Epoch 3
Accuracy: 73.0%, Avg loss: 0.552590
--------------------
Epoch 4
Accuracy: 73.0%, Avg loss: 0.547855
--------------------
Epoch 5
Accuracy: 73.0%, Avg loss: 0.543791
--------------------
Epoch 6
Accuracy: 73.1%, Avg loss: 0.544132
--------------------
Epoch 7
Accuracy: 73.1%, Avg loss: 0.545015
--------------------
Epoch 8
Accuracy: 73.0%, Avg loss: 0.548455
--------------------
Epoch 9
Accuracy: 73.2%, Avg loss: 0.541830
--------------------
Epoch 10
Accuracy: 73.2%, Avg loss: 0.542092
--------------------
Epoch 11
Accuracy: 73.0%, Avg loss: 0.542780
--------------------
Epoch 12
Accuracy: 73.2%, Avg loss: 0.542478
--------------------
Epoch 13
Accuracy: 73.2%, Avg loss: 0.543488
--------------------
Epoch 14
Accuracy: 73.0%, Avg loss: 0.546975
--------------------
Epoch 15
Accuracy: 72.9%, Avg loss: 0.544105


In [22]:
# Instance Feed Forward with Random Partially Connected
ffrpc_nn = FeedForwardRandomPartiallyConnected().to(device)
optimizer = torch.optim.AdamW(ffrpc_nn.parameters(), lr=1e-3)

# Train model
for t in range(epochs):
    print(f"--------------------\nEpoch {t+1}")
    train(train_dataloader, ffrpc_nn, loss_fn, optimizer)
    test(test_dataloader, ffrpc_nn, loss_fn)

--------------------
Epoch 1
Accuracy: 66.4%, Avg loss: 881107.765847
--------------------
Epoch 2
Accuracy: 54.8%, Avg loss: 520283.517180
--------------------
Epoch 3
Accuracy: 59.4%, Avg loss: 186885.609671
--------------------
Epoch 4
Accuracy: 64.2%, Avg loss: 63185.703227
--------------------
Epoch 5
Accuracy: 54.7%, Avg loss: 93801.947923
--------------------
Epoch 6
Accuracy: 64.1%, Avg loss: 20367.961752
--------------------
Epoch 7
Accuracy: 54.4%, Avg loss: 28702.595592
--------------------
Epoch 8
Accuracy: 65.7%, Avg loss: 5575.970821
--------------------
Epoch 9
Accuracy: 62.5%, Avg loss: 1778.799014
--------------------
Epoch 10
Accuracy: 53.1%, Avg loss: 357.225079
--------------------
Epoch 11
Accuracy: 58.3%, Avg loss: 56.395546
--------------------
Epoch 12
Accuracy: 53.4%, Avg loss: 14.871117
--------------------
Epoch 13
Accuracy: 52.0%, Avg loss: 7.734508
--------------------
Epoch 14
Accuracy: 51.1%, Avg loss: 3.973116
--------------------
Epoch 15
Accuracy: 50.6

In [23]:
# Instance Autoencoder with Wasp Waist
aeww_nn = AutoencoderWaspWaist().to(device)
optimizer = torch.optim.AdamW(aeww_nn.parameters(), lr=1e-3)

# Train model
for t in range(epochs):
    print(f"--------------------\nEpoch {t+1}")
    train(train_dataloader, aeww_nn, loss_fn, optimizer)
    test(test_dataloader, aeww_nn, loss_fn)

--------------------
Epoch 1
Accuracy: 73.1%, Avg loss: 0.546339
--------------------
Epoch 2
Accuracy: 73.0%, Avg loss: 0.544964
--------------------
Epoch 3
Accuracy: 72.9%, Avg loss: 0.552853
--------------------
Epoch 4
Accuracy: 73.2%, Avg loss: 0.542828
--------------------
Epoch 5
Accuracy: 73.1%, Avg loss: 0.544149
--------------------
Epoch 6
Accuracy: 73.2%, Avg loss: 0.543043
--------------------
Epoch 7
Accuracy: 73.1%, Avg loss: 0.542268
--------------------
Epoch 8
Accuracy: 73.1%, Avg loss: 0.542327
--------------------
Epoch 9
Accuracy: 73.2%, Avg loss: 0.542246
--------------------
Epoch 10
Accuracy: 73.3%, Avg loss: 0.542790
--------------------
Epoch 11
Accuracy: 73.1%, Avg loss: 0.542405
--------------------
Epoch 12
Accuracy: 73.0%, Avg loss: 0.545172
--------------------
Epoch 13
Accuracy: 73.1%, Avg loss: 0.543974
--------------------
Epoch 14
Accuracy: 73.1%, Avg loss: 0.548294
--------------------
Epoch 15
Accuracy: 73.0%, Avg loss: 0.544731


In [26]:
# Instance Autoencoder with Wasp Waist
saeww_nn = SmallAutoencoderWaspWaist().to(device)
optimizer = torch.optim.AdamW(saeww_nn.parameters(), lr=1e-3)

# Train model
for t in range(epochs):
    print(f"--------------------\nEpoch {t+1}")
    train(train_dataloader, saeww_nn, loss_fn, optimizer)
    test(test_dataloader, saeww_nn, loss_fn)

--------------------
Epoch 1
Accuracy: 72.8%, Avg loss: 0.548862
--------------------
Epoch 2
Accuracy: 72.8%, Avg loss: 0.546174
--------------------
Epoch 3
Accuracy: 72.9%, Avg loss: 0.543738
--------------------
Epoch 4
Accuracy: 73.2%, Avg loss: 0.543328
--------------------
Epoch 5
Accuracy: 73.1%, Avg loss: 0.543945
--------------------
Epoch 6
Accuracy: 72.9%, Avg loss: 0.543161
--------------------
Epoch 7
Accuracy: 73.0%, Avg loss: 0.543220
--------------------
Epoch 8
Accuracy: 73.1%, Avg loss: 0.545392
--------------------
Epoch 9
Accuracy: 73.0%, Avg loss: 0.543504
--------------------
Epoch 10
Accuracy: 73.1%, Avg loss: 0.546305
--------------------
Epoch 11
Accuracy: 72.9%, Avg loss: 0.542224
--------------------
Epoch 12
Accuracy: 73.1%, Avg loss: 0.547388
--------------------
Epoch 13
Accuracy: 73.1%, Avg loss: 0.543170
--------------------
Epoch 14
Accuracy: 72.9%, Avg loss: 0.542687
--------------------
Epoch 15
Accuracy: 73.1%, Avg loss: 0.542867


In [25]:
# Instance Autoencoder with Bottle Neck
aebn_nn = AutoencoderBottleNeck().to(device)
optimizer = torch.optim.AdamW(aebn_nn.parameters(), lr=1e-3)

# Train model
for t in range(epochs):
    print(f"--------------------\nEpoch {t+1}")
    train(train_dataloader, aebn_nn, loss_fn, optimizer)
    test(test_dataloader, aebn_nn, loss_fn)

--------------------
Epoch 1
Accuracy: 72.8%, Avg loss: 0.548678
--------------------
Epoch 2
Accuracy: 73.0%, Avg loss: 0.545284
--------------------
Epoch 3
Accuracy: 73.2%, Avg loss: 0.544128
--------------------
Epoch 4
Accuracy: 73.3%, Avg loss: 0.545803
--------------------
Epoch 5
Accuracy: 73.2%, Avg loss: 0.543304
--------------------
Epoch 6
Accuracy: 73.1%, Avg loss: 0.544070
--------------------
Epoch 7
Accuracy: 72.8%, Avg loss: 0.543895
--------------------
Epoch 8
Accuracy: 73.2%, Avg loss: 0.547800
--------------------
Epoch 9
Accuracy: 73.1%, Avg loss: 0.545578
--------------------
Epoch 10
Accuracy: 73.1%, Avg loss: 0.542680
--------------------
Epoch 11
Accuracy: 72.9%, Avg loss: 0.543640
--------------------
Epoch 12
Accuracy: 73.0%, Avg loss: 0.544538
--------------------
Epoch 13
Accuracy: 72.6%, Avg loss: 0.548037
--------------------
Epoch 14
Accuracy: 73.0%, Avg loss: 0.545278
--------------------
Epoch 15
Accuracy: 73.1%, Avg loss: 0.543019
